In [7]:
import os
import heapq
import googleapiclient.discovery

API_KEY = os.getenv("YOUTUBE_API_KEY")

In [8]:
import pandas as pd

In [9]:
news_titles = pd.read_json("./news_sentiment_data_20250220_115715.json")
news_titles.head(3)

,ticker,title,url,time_published,authors,summary,source,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,topics,collection_timestamp
0,AAPL,Can the Latest iPhone 16e Push the Apple Stock...,https://www.zacks.com/stock/news/2418900/can-t...,20250220T163200,[Aniruddha Ganguly],Although AAPL's AI push and growing Services b...,Zacks Commentary,0.296749,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.0870...","[{'topic': 'Retail & Wholesale', 'relevance_sc...",2025-02-20T19:57:09.571019
1,AAPL,Apple Stock Set For Streaming Lift? 'Severance...,https://www.benzinga.com/general/entertainment...,20250220T161828,[Chris Katje],"AppleTV+ has a streaming hit with ""Severance.""...",Benzinga,0.219397,Somewhat-Bullish,"[{'ticker': 'NFLX', 'relevance_score': '0.0525...","[{'topic': 'Technology', 'relevance_score': '1...",2025-02-20T19:57:09.571019
2,AAPL,"LKQ Q4 Earnings Surpass Expectations, Revenues...",https://www.zacks.com/stock/news/2418843/lkq-q...,20250220T155300,[Zacks Equity Research],LKQ reports mixed fourth-quarter results and e...,Zacks Commentary,0.056637,Neutral,"[{'ticker': 'DAN', 'relevance_score': '0.18625...","[{'topic': 'Earnings', 'relevance_score': '0.9...",2025-02-20T19:57:09.571019


In [4]:
news_titles["ticker"].unique()

array(['AAPL', 'NVDA', 'MSFT', 'AMZN', 'GOOG'], dtype=object)

In [18]:
def youtube_search(query, max_results=10):

    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=API_KEY)
    
    # Call the search.list method to retrieve results matching the query
    request = youtube.search().list(
        part="snippet",
        q=query,
        type="video",
        maxResults=max_results
    )
    response = request.execute()
    
    # Extract video IDs, titles, and publication dates from the response
    videos = []
    for item in response.get("items", []):
        video_data = {
            "video_id": item["id"]["videoId"],
            "title": item["snippet"]["title"],
            "published_at": item["snippet"]["publishedAt"]
        }
        videos.append(video_data)
    
    return videos


def get_video_comments(video_id, max_results=100):
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=API_KEY)

    comments = []
    next_page_token = None

    while len(comments) < max_results:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=min(100, max_results - len(comments)),
            textFormat="plainText",
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response.get("items", []):
            comment = item["snippet"]["topLevelComment"]["snippet"]
            comment_data = {
                "author": comment["authorDisplayName"],
                "text": comment["textDisplay"],
                "likes": comment["likeCount"],
                "published_at": comment["publishedAt"]
            }
            comments.append(comment_data)

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return comments

In [19]:
news_titles.loc[0]["title"]

'Can the Latest iPhone 16e Push the Apple Stock Higher in 2025?'

In [25]:
parsed_video_data = []
for ticker, title in zip(news_titles["ticker"].values[: 3], news_titles["title"].values[: 3]):
    for ticker_data in youtube_search(query=title, max_results=10):
        comments = get_video_comments(ticker_data["video_id"], max_results=500)
        top_comments = heapq.nlargest(500, comments, key=lambda item: item["likes"])
        ticker_data["title"] = title
        ticker_data["ticker"] = ticker
        ticker_data["top_comments"] = top_comments
        parsed_video_data.append(ticker_data)

In [26]:
parsed_video_data[0]

{'video_id': 'Gvx5SwB6trE',
 'title': 'Can the Latest iPhone 16e Push the Apple Stock Higher in 2025?',
 'published_at': '2025-02-19T22:34:06Z',
 'ticker': 'AAPL',
 'top_comments': [{'video_id': 'Gvx5SwB6trE',
   'author': '@O-jack1',
   'text': "Tech companies will still be a key market driver. I own a handful of them worth $200,000. But my trust level in Trump is so low that I would not commit for too long. I'm at crossroad if I should sell now or not, my goal is at least $1m before retirement. I need advice on this.",
   'likes': 33,
   'published_at': '2025-02-20T12:08:11Z'},
  {'video_id': 'Gvx5SwB6trE',
   'author': '@volitiveclover',
   'text': 'she does not know what she’s talking about',
   'likes': 6,
   'published_at': '2025-02-19T23:15:20Z'},
  {'video_id': 'Gvx5SwB6trE',
   'author': '@ljacobs357',
   'text': "I don't get the appeal of the iPhone, when there are car better options.",
   'likes': 0,
   'published_at': '2025-02-20T13:40:20Z'},
  {'video_id': 'Gvx5SwB6trE',
 

In [10]:
def get_video_comments(video_id, max_results=100):
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=API_KEY)

    comments = []
    next_page_token = None

    while len(comments) < max_results:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=min(100, max_results - len(comments)),
            textFormat="plainText",
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response.get("items", []):
            comment = item["snippet"]["topLevelComment"]["snippet"]
            comment_data = {
                "video_id": video_id,
                "author": comment["authorDisplayName"],
                "text": comment["textDisplay"],
                "likes": comment["likeCount"],
                "published_at": comment["publishedAt"]
            }
            comments.append(comment_data)

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return comments


# video on AAPL earnings from 2 weeks ago
video_id = "u_D2eCtnFTs"
comments = get_video_comments(video_id, max_results=500)
print(f"Downloaded {len(comments)} comments.")

Downloaded 168 comments.


In [11]:
heapq.nlargest(7, comments, key=lambda item: item["likes"])

[{'video_id': 'u_D2eCtnFTs',
  'author': '@KingTechHD',
  'text': '😂 that’s crazy. Once it realizes it’s not the MagSafe Battery Pack it stops charging. Unfortunately reverse charging is slow & terrible. Apple wants to get it right when they enable it',
  'likes': 538,
  'published_at': '2022-11-04T17:49:57Z'},
 {'video_id': 'u_D2eCtnFTs',
  'author': '@ReviewswBryz',
  'text': 'I feels like apple already have most of these hardware capabilities and they will enable them at their own convenient time as a new release 😁',
  'likes': 308,
  'published_at': '2022-11-04T18:42:51Z'},
 {'video_id': 'u_D2eCtnFTs',
  'author': '@leilinatoilolo3649',
  'text': 'I think that it would be very useful if they enabled it sooner rather than later.',
  'likes': 139,
  'published_at': '2022-11-04T17:38:18Z'},
 {'video_id': 'u_D2eCtnFTs',
  'author': '@PaulRybicki83',
  'text': 'So pretty much it can’t',
  'likes': 56,
  'published_at': '2022-11-04T17:48:32Z'},
 {'video_id': 'u_D2eCtnFTs',
  'author': '@